export cells as modules?



In [2]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var moduleOutput = PROFILE_PATH + '/jupytangular2/Utilities/.modules';
var importer = require('../Core');
var fs = require('fs');
var path = require('path');

var exportNotebook = (notebook) => {
    const name = path.basename(notebook).replace(/\.ipynb/ig, '');
    const parent = path.basename(path.dirname(notebook));
    if(!fs.existsSync(moduleOutput)) {
        fs.mkdirSync(moduleOutput);
    }
    if(!fs.existsSync(path.join(moduleOutput, parent))) {
        fs.mkdirSync(path.join(moduleOutput, parent));
    }
    if(!fs.existsSync(path.join(moduleOutput, parent, name))) {
        fs.mkdirSync(path.join(moduleOutput, parent, name));
    }
    
    return importer.getCells(notebook, ['*', 'code', 'markdown'])
    .then(cells => {
        const results = [];
        for(const i in cells) /* exclude this cell */ {
            if(!cells.hasOwnProperty(i)) {
                continue;
            }
            var extension;
            if(cells[i].cell_type === 'markdown') {
                extension = '.md';
            } else {
                extension = '.js';
            }
            const cellPath = path.join(moduleOutput, parent, name,
                                       'cell-' + i + extension);
            fs.writeFileSync(cellPath, cells[i].source.join(''));
            results.push(cellPath);
        }
        return results;
    })
}
module.exports = exportNotebook;


[Function: exportNotebook]

export all notebooks?



In [3]:
var glob = require('glob');
var path = require('path');
var importer = require('../Core');

var exportAll = (project) => {
    const notebooks = glob.sync('**/*.ipynb', {cwd: project});
    return importer.runAllPromises(notebooks.map(n => resolve => {
        return exportNotebook(path.join(project, n))
        .then(r => resolve(r))
        .catch(e => { console.log(e); resolve(); })
    }))
}
module.exports = exportAll;


[Function: exportAll]

test notebook export



In [4]:
var importer = require('../Core');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/jupytangular2';

$$.async()
importer.import('notebook.ipynb[export cells modules]')
.then(exportNotebook => importer.import(
    'notebook.ipynb[export all notebooks]', {exportNotebook}))
.then(exportAll => exportAll(PROJECT_PATH))
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e))


[ '/Users/briancullinan/jupytangular2/Utilities/.modules/Analytics/budgeting/cell-0.md',
  '/Users/briancullinan/jupytangular2/Utilities/.modules/Analytics/code analysis/cell-0.md',
  '/Users/briancullinan/jupytangular2/Utilities/.modules/Analytics/reconcile timeline data/cell-0.md',
  '/Users/briancullinan/jupytangular2/Utilities/.modules/Analytics/reconcile timeline data/cell-1.js',
  '/Users/briancullinan/jupytangular2/Utilities/.modules/Analytics/reconcile timeline data/cell-2.js',
  '/Users/briancullinan/jupytangular2/Utilities/.modules/Analytics/reconcile timeline data/cell-3.js',
  '/Users/briancullinan/jupytangular2/Utilities/.modules/Analytics/reconcile timeline data/cell-4.js',
  '/Users/briancullinan/jupytangular2/Utilities/.modules/Analytics/socializing/cell-0.md',
  '/Users/briancullinan/jupytangular2/Utilities/.modules/Core/import notebook/cell-0.md',
  '/Users/briancullinan/jupytangular2/Utilities/.modules/Core/import notebook/cell-1.js',
  '/Users/briancullinan/jupytang

import all cell modules?

In [19]:
var fs = require('fs');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Documents/jupytangular2/Utilities/.modules';
var project = PROFILE_PATH + '/Documents/jupytangular2';

var cells = glob.sync('**/cell-*', {cwd: PROJECT_PATH});
for(const c of cells) {
    const cell = path.basename(c);
    const notebook = path.basename(path.dirname(c));
    const parent = path.basename(path.dirname(path.dirname(c)));
    if(parent === 'jupytangular2') {
        continue;
    }
    const nb = JSON.parse(fs.readFileSync(path.join(project, parent, notebook + '.ipynb')));
    let counter = 0;
    for(const i in nb.cells) {
        if(!nb.cells.hasOwnProperty(i)) {
            continue;
        }
        if(nb.cells[i].cell_type === 'code') {
            if(cell === 'cell-' + counter + '.js') {
                nb.cells[i].source = fs.readFileSync(path.join(PROJECT_PATH, c)).toString().split('\n');
            }
            counter++;
        }
    }
    fs.writeFileSync(path.join(project, parent, notebook + '.ipynb'), JSON.stringify(nb, null, 2));
}



import module as notebook cell?

TODO:
import gist as notebook
import random instructions as notebook
